In [1]:
from lightfm_recomender import LightfmRecomender
import os
%load_ext autoreload
%autoreload 2
new_directory = '/Users/vasevooo/projects/GameBuddy'
os.chdir(new_directory)
import pandas as pd 
import requests
import json
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np
from lightfm import LightFM
from scipy.sparse import csr_matrix, save_npz, load_npz
import re
from bs4 import BeautifulSoup

/Users/vasevooo/projects/GameBuddy/steam/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/vasevooo/projects/GameBuddy/steam/lib/python3.9/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


### обновление предобработанных данных для модели 

In [7]:
games = pd.read_csv('data/game_info_with_scores.csv')
item_dict ={}
games = games[['appid', 'Name']].sort_values('appid').reset_index()
games['appid'] = games['appid'].astype(int)

for i in range(games.shape[0]):
    item_dict[int(games.loc[i,'appid'])] = games.loc[i,'Name']

In [8]:
## сохранение словаря по играм для дальнейшего использования

with open("data/for_models/item_dict.json", "w") as outfile:
    json.dump(item_dict, outfile, indent=4)

### Тест класса LigthFM Recomender

In [2]:
item_dict_path = 'data/for_models/item_dict.json'
gametime_df_path = 'data/final_gametime_df.csv'

lfr = LightfmRecomender(gametime_df_path, item_dict_path)

In [25]:
steam_url = 'https://steamcommunity.com/id/momuss/'

In [31]:
new_user, steamid = lfr.get_user_games(steam_url)

In [32]:
matrix, df = lfr.get_csr_matrix_for_all_users(new_user)

In [33]:
## в приложении поставить 100 эпох. 1 для теста

model = lfr.fit_model(matrix, epochs=1)

In [34]:
lfr.sample_recommendation_user(model, df, steamid, threshold = 0, nrec_items = 10)

User: 76561197994384567
Recommended Items:
1- PAYDAY 2
2- Portal 2
3- The Witcher® 3: Wild Hunt
4- Terraria
5- Half-Life 2
6- Garry's Mod
7- The Elder Scrolls V: Skyrim
8- Dying Light
9- Tomb Raider
10- Don't Starve Together

 Known Likes:
1- Sea of Thieves 2023 Edition
2- Dota Underlords
3- Tropico 6
4- Human: Fall Flat
5- PAYDAY™ The Heist
6- Warhammer® 40,000: Dawn of War® II Chaos Rising
7- Command & Conquer: Red Alert 3
8- Call of Duty®: Modern Warfare® 2 (2009)
9- Call of Duty®: Modern Warfare® 2 (2009)
10- Supreme Commander: Forged Alliance
11- Sid Meier's Civilization® V
12- Warhammer® 40,000: Dawn of War® - Game of the Year Edition
13- Killing Floor
14- Counter-Strike: Global Offensive
15- Left 4 Dead 2
16- Left 4 Dead
17- Counter-Strike: Source
18- Counter-Strike


### cold start

In [23]:
df_cold_start = pd.DataFrame({'game_names': ['Counter-Strike', 'Team Fortress Classic', 
                                             'Day of Defeat', 
                                             'Half-Life: Source', 'Half-Life: Opposing Force'],
                   'rankings': [5, 5, 8, 7, 3]})

In [24]:
df_cold_start.head(2)

,game_names,rankings
0,Counter-Strike,5
1,Team Fortress Classic,5


In [35]:
cold_start_user, steamid_cold = lfr.get_user_games_cold_start(df_cold_start)

In [36]:
matrix_cs, df_cs = lfr.get_csr_matrix_for_all_users(cold_start_user)

In [20]:
## в приложении поставить 100 эпох. 1 для теста
model = lfr.fit_model(matrix_cs, epochs=1)

In [38]:
lfr.sample_recommendation_user(model, df_cs, steamid_cold, threshold = 0, nrec_items = 10)

User: 1234567891010102930
Recommended Items:
1- Counter-Strike: Global Offensive
2- Left 4 Dead 2
3- PAYDAY 2
4- Portal 2
5- The Witcher® 3: Wild Hunt
6- Terraria
7- Half-Life 2
8- Garry's Mod
9- The Elder Scrolls V: Skyrim
10- Dying Light

 Known Likes:
1- Half-Life: Source
2- Day of Defeat
3- Team Fortress Classic
4- Counter-Strike
